# Chains
* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [3]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [4]:
# !pip install pandas

In [5]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,"""I ordered a king size set. My only criticism..."
1,Waterproof Phone Pouch,"""I loved the waterproof sac although the open..."
2,Luxury Air Mattress,"""This mattress had a small hole in the top of..."
3,Pillows Insert,"""This is the best throw pillow fillers on Ama..."
4,Milk Frother Handheld,"""I loved this product. But they only seem to ..."


## LLMChain

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [9]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [10]:
chain = LLMChain(llm=llm, prompt=prompt)

In [11]:
product1 = "Queen Size Sheet Set"
chain.run(product1)

'Royal Sheets Co.'

In [12]:
product2 = "Milk Frother Handheld"
chain.run(product2)

'"Frothmaster" or "Milk Froth Handheld" would be suitable names for a company that makes milk frother handhelds.'

## SimpleSequentialChain

In [13]:
from langchain.chains import SimpleSequentialChain

In [14]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [15]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [17]:
overall_simple_chain.run(product1)



> Entering new SimpleSequentialChain chain...
Regal Rest.
Regal Rest offers high-quality, luxurious, and comfortable bedding solutions to ensure a good night's sleep for their customers.

> Finished chain.


"Regal Rest offers high-quality, luxurious, and comfortable bedding solutions to ensure a good night's sleep for their customers."

In [18]:
overall_simple_chain.run(product2)



> Entering new SimpleSequentialChain chain...
FrothWorks.
FrothWorks is a specialty coffee shop that serves freshly brewed coffee with a rich and creamy froth, creating a unique taste experience.

> Finished chain.


'FrothWorks is a specialty coffee shop that serves freshly brewed coffee with a rich and creamy froth, creating a unique taste experience.'

## SequentialChain

In [19]:
from langchain.chains import SequentialChain

In [20]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [21]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [22]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )

In [23]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [29]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [30]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': ' "Je trouve le goût médiocre. La mousse ne tient pas c\'est bizarre. J\'achète les mêmes dans le commerce et le goût est bien meilleur...Vieux lot ou contrefaçon !?"',
 'English_Review': '"I find the taste mediocre. The foam doesn\'t hold, it\'s weird. I buy the same ones in stores and the taste is much better...Old batch or counterfeit!?"',
 'summary': "The taste is mediocre and the foam doesn't hold, making the reviewer suspect that it may be an old or counterfeit batch.",
 'followup_message': "Réponse de suivi :\n\nLa saveur est médiocre et la mousse ne tient pas, ce qui laisse supposer au critique qu'il s'agit peut-être d'un lot ancien ou contrefait. Il est important de se méfier des produits qui ne sont pas frais, car ils peuvent altérer la qualité de la boisson. Il est également important de vérifier la provenance et l'authenticité des produits que nous achetons. Nous devrions toujours être conscients de ce que nous consommons afin de garantir une expérience gustative

In [32]:
review = df.Review[4]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Review': '\xa0"I loved this product. But they only seem to last a few months. The company was great replacing the first one (the frother falls out of the handle and can\'t be fixed). The after 4 months my second one did the same. I only use the frother for coffee once a day. It\'s not overuse or abuse. I\'m very disappointed and will look for another. As I understand they will only replace once. Anyway if you have one good luck."',
 'English_Review': "I really enjoyed using this product, but unfortunately, it seems to only last a few months. The company was great about replacing the first one when the frother fell out of the handle and couldn't be fixed, but then after four months, my second one did the same thing. I only use the frother once a day for coffee, so it's not like I'm overusing or abusing it. I'm really disappointed and will have to look for another one to buy. Plus, I've heard they will only replace it once, so that's not very reassuring. Anyway, good luck if you decide

## Router Chain

In [33]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [34]:

prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]


In [35]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [36]:
llm = ChatOpenAI(temperature=0, model=llm_model)

In [37]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [38]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [39]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [40]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [41]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [42]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an object that absorbs all radiation that falls on it and emits radiation at all wavelengths. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding the behavior of stars, as well as in the development of technologies such as incandescent light bulbs and infrared cameras."

In [45]:
chain.run("Whats the root of the conflict between Israel and Palestine?")



> Entering new MultiPromptChain chain...
History: {'input': 'Whats the root of the conflict between Israel and Palestine?'}
> Finished chain.


'As an AI language model, I do not have personal beliefs or opinions. However, the conflict between Israel and Palestine has its roots in a complex history of religious, cultural, and political factors. The conflict dates back to the late 19th century when Jewish immigrants began to settle in Palestine, then under Ottoman rule. After World War I, the British took control of Palestine and promised both Jews and Arabs self-determination. In 1947, the United Nations voted to partition Palestine into separate Jewish and Arab states, but this plan was rejected by the Arab states, leading to a war between Israel and its Arab neighbors. Since then, there have been numerous conflicts and negotiations, with both sides claiming the right to the land and sovereignty over Jerusalem. The conflict remains unresolved to this day.'

In [46]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA carries the genetic information that determines the characteristics and functions of each cell. DNA contains the instructions for the synthesis of proteins, which are essential for the structure and function of cells. Additionally, DNA is responsible for the transmission of genetic information from one generation to the next. Therefore, every cell in our body needs DNA to carry out its specific functions and to maintain the integrity of the organism as a whole.'